In [1]:
from pathlib import Path

import numpy as np
import pyvista as pv

from cardiac_electrophysiology.ulac import preprocessing as pp

# Load Dataset

In [2]:
patient_id = "patient_01"
infile_name = Path(f"../data/raw/{patient_id}_mesh_with_uacs_fibers_tags.vtk")
outfile_name = Path(f"../data/processed/{patient_id}/mesh_with_fibers_tags.vtk")
legacy_anatomical_tags = {
    "Body": 11,
    "LAA": 13,
    "LIPV": 21,
    "LSPV": 23,
    "RIPV": 25,
    "RSPV": 27,
}

new_anatomical_tags = {
    "Body": 0,
    "LAA": 1,
    "LIPV": 2,
    "LSPV": 3,
    "RIPV": 4,
    "RSPV": 5,
}

input_mesh = pv.read(infile_name)
triangular_mesh = pp.convert_unstructured_to_polydata_mesh(input_mesh)
del triangular_mesh.point_data["alpha"]
del triangular_mesh.point_data["beta"]
for key, value in legacy_anatomical_tags.items():
    anatomical_region = np.where(triangular_mesh.cell_data["anatomical_tags"] == value)[0]
    triangular_mesh.cell_data["anatomical_tags"][anatomical_region] = new_anatomical_tags[key]

In [3]:
coarse_mesh = pp.coarsen_mesh(triangular_mesh, decimation_factor=0.9)
outfile_name.parent.mkdir(parents=True, exist_ok=True)
pv.save_meshio(outfile_name, coarse_mesh)

In [10]:
extracted_region = coarse_mesh.extract_values(new_anatomical_tags["RSPV"], scalars="anatomical_tags")

plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(coarse_mesh, color="lightgrey", opacity=0.3, style="wireframe")
plotter.add_mesh(
    extracted_region,
    scalars="anatomical_tags",
    cmap="tab20",
    show_scalar_bar=False,
)
plotter.show()

Widget(value='<iframe src="http://localhost:39347/index.html?ui=P_0x7f8cfc354190_5&reconnect=auto" class="pyvi…